In [1]:
### Loading required modules

import numpy as np
import tensorflow as tf
import os,sys
import datetime
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, save_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import cv2
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

tf.config.list_physical_devices('GPU')

path="/home/ubuntu/pediatric_dental_age_prediction/raw/"

print("Loaded all modules")

2023-11-15 11:07:46.521573: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 11:07:47.176333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loaded all modules


2023-11-15 11:07:48.616195: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-15 11:07:48.640405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-15 11:07:48.642449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
### Data augmentation of raw panoramic images 
### By rotating, shifting, zooming, flipping, brightness-adjusting


def rawImgAugment(mode, numAugPerImg):
    datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.05,
        zoom_range=0.15,
        horizontal_flip=True,
        brightness_range=[0.7,1.1],
        fill_mode='nearest'
        )
    
    
    print("Original files: {0}".format(path+mode))
    print("Saving to ... : {0}".format(path+mode+"_aug"))
    
    for name_img in os.listdir(path+mode):
        img = cv2.imread(path+mode+"/{0}".format(name_img))  # cv2 img
        res = cv2.resize(img, dsize=(512, 256)) ### Resizing to 
        
        res_array = img_to_array(res)
        res_array = res_array.reshape((1,) + res_array.shape)
    
        i = 0
    
        for batch in datagen.flow(res_array, batch_size=1,shuffle=True,
                                  save_to_dir=path+mode+"_aug", save_prefix=name_img.replace(".png",""),
                                        save_format="jpeg"):
            i += 1
            if i > numAugPerImg:
                break
    
    print("Data augmentation done for {0} set with {1} per each image".format(mode, numAugPerImg))

rawImgAugment("train", 30)   # Augmenting 30 images per each train image
rawImgAugment("test", 10)    # Augmenting 10 images per each test image

Original files: /home/ubuntu/project/raw_test/train
Saving to ... : /home/ubuntu/project/raw_test/train_aug
Data augmentation done for train set with 30 per each image
